### Import Libraries

In [1]:
"If it does not work use code below while running this notebook in this dir."
# import sys
# import os
# sys.path.append(os.path.abspath(os.getcwd()))

from mytorch import Tensor, Model
from mytorch import activation as active_func
from mytorch import loss as loss_func
from mytorch import optimizer as optim 
from mytorch import layer as nn
from mytorch.util import DataLoader

from mytorch.util import flatten

import matplotlib.pyplot as plt

### Load MNIST

In [2]:
"""
load data set with given data loader.
you have 10000 train data, 1000 for each number, remember to shuffle training data.
you have 1000 test data, 100 for each number.

loaded data is a list of (img, label)
type of img is Tensor.

TODO: you have to get this list and create batches for training.
you can also apply this changes later in the Training part for convenience.
"""
data_loader = DataLoader(train_addr='MNIST_light/MNIST/train', test_addr='MNIST_light/MNIST/test')
data_loader.load(train_batch_size=64, test_batch_size=64)
"you can see how data is loaded"
# print(data_loader.getTrain()[0][0].shape)
# print(data_loader.getTrain()[0][1].shape)

loading train...
loading test...
processing...


'you can see how data is loaded'

### Define and Create your Model

In [3]:
class MyModel(Model):
    def __init__(self):
        super(Model, self).__init__()
        "TODO: define your layers. order is important" 
        self.fc1 = nn.Linear(28 * 28, 256)
        self.relu = active_func.relu
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        self.softmax = active_func.softmax

    def forward(self, x: Tensor) -> Tensor:
        "TODO: define forward pass"
        x = flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.softmax(self.fc4(x))
        return x

model = MyModel()
print(model.summary())

fc1:  linear - total param: 200704 - in: 784, out: 256
fc2:  linear - total param: 32768 - in: 256, out: 128
fc3:  linear - total param: 8192 - in: 128, out: 64
fc4:  linear - total param: 640 - in: 64, out: 10
None


### Choose a Loss Function

In [4]:
"TODO: choose a proper loss function"

criterion = loss_func.CategoricalCrossEntropy

### Choose an Optimizer

In [5]:
"TODO: choose a proper optimizer"
optimizer = optim.SGD(model.parameters(), learning_rate=0.001)

### Training methods

In [ ]:
# just use it for argmax
import numpy as np

def train_one_epoch(model: Model, data_loader, optimizer: optim, criterion, length):
  acc = 0
  for batch in data_loader.getTrain():
      x_train_dev = batch[0]
      y_train_dev = batch[1]

      y_pred_dev = model(x_train_dev)

      loss = criterion(y_pred_dev, y_train_dev)

      loss.backward()

      optimizer.step()

      yp= np.argmax(y_pred_dev.data, axis=1)
      acc += np.sum(yp == y_train_dev.data)

  return loss.data, acc/length



def caluculate_acc(data_loader, model, length):
  acc = 0
  for batch in data_loader.getTest():
    x_test_dev = batch[0]
    y_test_dev = batch[1]
    yp = model(x_test_dev)

    yp= np.argmax(y_test_dev.data, axis=1)
    acc += np.sum(yp == y_test_dev.data)
  return acc / length

### Training Model

In [6]:
"TODO: define number of epoch and train batches of data on your model. also test each epoch."
EPOCH = 13

for i in range(EPOCH):

    train_acc = []
    test_acc = []
    best_train_acc = 0.0
    best_test_acc = 0.0

    "TODO: train over your defined batches and save train accuracy for each epoch."
    # train_acc.append(train_one_epoch(model, data_loader, optimizer, criterion, 1000)[1].item())
    train_acc.append(train_one_epoch(model, data_loader, optimizer, criterion, 1000)[1])
    
    if best_train_acc < best_train_acc[-1]:
        best_train_acc = train_acc[-1]

    "TODO: test your model after each training and save test accuracy for each epoch."

    # test_acc.append(caluculate_acc(data_loader, model, 1000).item())
    test_acc.append(caluculate_acc(data_loader, model, 1000))


    if best_test_acc < test_acc[-1]:
        best_test_acc = test_acc[-1]

    if train_acc[-1] > 0.99 and test_acc[-1] > 0.99:
        break


result: [ 7.14996927e+01 -2.27215756e+01 -2.09676304e+02 -1.13274043e+02
 -1.44832515e+02  1.30782977e+01  7.03504052e+01  1.23276134e+01
  1.29744721e+02  2.91267829e+01 -1.79743489e+00  6.51854032e+01
 -2.00854714e+01 -1.24183725e+02 -8.14958921e+01  1.18776262e+02
  1.29466088e+02 -1.35864046e+02  3.68693578e+01  1.80371389e+02
  3.46773201e+01 -1.05760406e+02 -1.23053620e+02  1.94003154e+01
 -1.96859741e+02 -4.04898355e+01 -1.02094698e+02  3.71761183e+01
  5.51158556e+00  1.35906693e+02 -1.99723534e+02 -1.03805816e+02
  1.29952246e+02  2.02444672e+02 -1.28054186e+02  3.23738186e+01
  2.25988894e+01 -2.70646749e+00 -6.18850327e+01 -8.15232398e+00
  1.37524291e+02 -1.08798246e+01  3.36292769e+01  5.63985200e+01
  5.95692712e+01 -7.13068718e+00  9.06896255e+00 -1.32974020e+02
  1.32054801e+01  3.25994900e+01  7.72716361e+01 -2.88075263e+01
 -9.77915172e-01 -7.36225894e+00 -2.98525831e+01 -1.29278503e+02
  1.48215236e+01 -9.78257445e+01  1.91925242e+02 -2.49764929e+01
 -7.85071263e+01 

### Evaluate Model Performance

In [ ]:
plt.plot(train_acc, label='train accuracy')
plt.plot(test_acc, label = 'test accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

print("\nOn train - best accuracy: {:.2f}, final accuracy: {:.2f}".format(best_train_acc, train_acc[-1]))
print("On test - best accuracy: {:.2f}, final accuracy: {:.2f}".format(best_test_acc, test_acc[-1]))